In [ ]:
import numpy as np
from pathlib import Path
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal, fftpack
import time
import datetime
import h5py

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle


from src.data.data_utils import get_min_max, scaler
from src.features.build_features import create_fft
from src.visualization.visualize import create_time_frequency_plot, plot_freq_peaks
# for plotly, if wanted
# from plotly.subplots import make_subplots
# import plotly.graph_objects as go
# import plotly.io as pio

%matplotlib inline

# to clear outputs from cells
from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [ ]:
# set the root (parent folder) and the data folder locations
folder_root = Path.cwd().parent # get root folder of repository

folder_raw_data = folder_root / 'data/raw/IMS/' # raw data folder

In [ ]:
# load text file for first measurement
# first test folder location
folder_1st = folder_raw_data / '1st_test'

# can use numpy...
d = np.loadtxt(folder_1st / '2003.10.22.12.06.24')

# let's us pandas
    # b1_ch1 - Bearing 1, channel 1
    # b1_ch2 - Bearing 1, channel 2
    # etc, etc ...

col_names = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5', 'b3_ch6', 'b4_ch7', 'b4_ch8']
df = pd.read_csv(folder_1st / '2003.10.22.12.06.24', sep='\t', names=col_names)

In [ ]:
df.head()

In [ ]:
# what is the shape of the dataframe?
df.shape

Gousseau et al. postulate in their paper ['Analysis of the Rolling Element Bearing data set of the Center for Intelligent Maintenance Systems of theUniversity of Cincinnati'](https://hal.archives-ouvertes.fr/hal-01715193) that the actual collection frequency is 20.48 kHz.

We will use this collection frequency.

In [ ]:
# plot first bearing channel
fig, ax = plt.subplots()

ax.plot(
    np.arange(0,df.shape[0], dtype='float64') / (20.48 * 10**3), # make x-axis in seconds
    df['b1_ch1'] # acceleration data
)

In [ ]:
# practice detrending
fig, ax = plt.subplots(1, 1, figsize=(15, 8))

plt.plot(df['b1_ch1'], alpha=0.5, label='original signal')
y_detrend = signal.detrend(df['b1_ch1'], type="linear")
plt.plot(y_detrend, alpha=0.5, label='detrended signal')

# apply either a hamming or kaiser windowing function
# y_detrend *= np.hamming(len(y_detrend))
y_detrend *= np.kaiser(len(y_detrend), 3)
plt.plot(y_detrend, alpha=0.5, label='windowed signal')
plt.legend(loc='center left')

Use a function to create the FFT (`create_fft`), and another function to plot the time and frequency domains (`create_time_frequency_plot`).

In [ ]:
# create fft
x, y, xf, yf = create_fft(df, y_name='b1_ch2', sample_freq=20480.0, window='kaiser', beta=3)

# plot
create_time_frequency_plot(x, y, xf, yf, save_plot=False)

## Spectrogram
We want to build a spectrogram of the first test set.

First, get the name of all the files in the folder.

### Aside: Naming Each Sample

In [ ]:
date_list = sorted(os.listdir(folder_1st))
date_list[:10] # check to see it makes sense...

The names are a bit clunky. Let's convert them all to UNIX timestamps.

In [ ]:
# from https://stackoverflow.com/a/9637908/9214620
s = '2003.10.22.12.06.24'
t = time.mktime(datetime.datetime.strptime(s, "%Y.%m.%d.%H.%M.%S").timetuple())
print(t)

In [ ]:
s.replace('.', '_')

To make sure it makes sense, convert back to a readable format.

In [ ]:
print(datetime.datetime.fromtimestamp(t).strftime('%Y-%m-%d %H:%M:%S'))

### Calculate FFT for Each Signal
We'll calculate the FFT for each signal. This will be stored in a pandas dataframe, with each new column being a new signal.

In [ ]:
start_time = '2003.10.22.12.06.24'
start_time = time.mktime(datetime.datetime.strptime(start_time, "%Y.%m.%d.%H.%M.%S").timetuple())

In [ ]:
def build_spectrogram_df(folder, date_list, channel_name='b3_ch5', start_time='2003.10.22.12.06.24', col_day_increment=False,
                         col_names = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5', 'b3_ch6', 'b4_ch7', 'b4_ch8']):
    '''function that builds the spectrogram data'''
    
    # convert start_time to unix timestamp
    start_time = time.mktime(datetime.datetime.strptime(start_time, "%Y.%m.%d.%H.%M.%S").timetuple())

    # instantiate dataframe for the spectrogram
    dft = pd.DataFrame()
       
    # dictionary to store any labels
    labels_dict = {}

    # iterate through each date that samples were taken
    # date_list should be sorted from earliest to latest
    for i, sample_name in enumerate(date_list):
        # convert sample_name to unix timestamp
        unix_timestamp = time.mktime(datetime.datetime.strptime(sample_name, "%Y.%m.%d.%H.%M.%S").timetuple())
        date_nice_format = datetime.datetime.fromtimestamp(unix_timestamp).strftime('%Y-%m-%d %H:%M:%S') # reformat date

        # open the file containing the measurements
        df = pd.read_csv(folder / sample_name, sep='\t', names=col_names)

        # create fft
        xf, yf = create_fft(df, x_name='Time', y_name=channel_name, sample_freq=20480.0, show_plot=False, window='kaiser', beta=3)
        # xf, yf = create_fft(df, x_name='Time', y_name=channel_name, sample_freq=20000.0, show_plot=False, window='kaiser', beta=3)

        # change sample name slightly to change '.' to '_' (personal preference)
        sample_name = sample_name.replace('.', '_')

        # append the time increments
        time_increment_seconds = unix_timestamp-start_time
        time_increment_days = time_increment_seconds /(60 * 60 * 24)
        
        # create new column for the current sample_name FFT
        if col_day_increment == False:
            dft[date_nice_format] = yf
        if col_day_increment == True:
            dft[str(time_increment_days)] = yf

        # create new dictionary key and values to store lable info
        labels_dict[sample_name] = [date_nice_format, sample_name, unix_timestamp, time_increment_seconds, time_increment_days]

    dft = dft.set_index(xf, drop=True) # index as frequency (Hz)
    return dft, labels_dict

Let's look at bearing 1, channel 1. This bearing did not fail in this run.

In [ ]:
folder_1st = folder_raw_data / '1st_test'

date_list = sorted(os.listdir(folder_1st))

df_spec, labels_dict = build_spectrogram_df(folder_1st, date_list, channel_name='b1_ch1', start_time='2003.10.22.12.06.24')
df_spec.head()

Plot the spectrogram.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 10))

plt.pcolormesh(df_spec.columns, df_spec.index, df_spec)

ax.set_ylabel("Frequency (Hz)")
plt.xticks(df_spec.columns[::100]) # show every 100th date on x-axis ticks
plt.xticks(rotation=75)

plt.show()

The above spectrogram is fairly "dim". This is partly because the 1000 Hz peak is dominating the FFT.

We'll adjust the vmax to get better clarity.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 10))

# set vmax to 0.01
plt.pcolormesh(df_spec.columns, df_spec.index, df_spec, vmax=0.01)

ax.set_ylabel("Frequency (Hz)")
plt.xticks(df_spec.columns[::100]) # show every 100th date on x-axis ticks
plt.xticks(rotation=75)

plt.show()

That's better! You can see in the above spectrogram that as this bearing (bearing 1) gets closer to the end of its run, the higher frequencies get noisier and larger.

Now let's look at a bearing that did have a failure. Bearing 3 had an inner race defect.

In [ ]:
folder_1st = folder_raw_data / '1st_test'

date_list = sorted(os.listdir(folder_1st))

# reminder of bearing acceleration channels
# col_names = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5', 'b3_ch6', 'b4_ch7', 'b4_ch8']

df_spec, labels_dict = build_spectrogram_df(folder_1st, date_list, channel_name='b3_ch5', start_time='2003.10.22.12.06.24')

# plot spectrogram
fig, ax = plt.subplots(1, 1, figsize=(15, 10))

plt.pcolormesh(df_spec.columns, df_spec.index, df_spec, vmax=0.01)
ax.set_ylabel("Frequency (Hz)")
plt.xticks(df_spec.columns[::100]) # show every 100th date on x-axis ticks
plt.xticks(rotation=75)

plt.show()

In [ ]:
folder_1st = folder_raw_data / '1st_test'

date_list = sorted(os.listdir(folder_1st))

# reminder of bearing acceleration channels
# col_names = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5', 'b3_ch6', 'b4_ch7', 'b4_ch8']

df_spec, labels_dict = build_spectrogram_df(folder_1st, date_list, channel_name='b3_ch6', start_time='2003.10.22.12.06.24')

# plot spectrogram
fig, ax = plt.subplots(1, 1, figsize=(15, 10))

plt.pcolormesh(df_spec.columns, df_spec.index, df_spec, vmax=0.01)
ax.set_ylabel("Frequency (Hz)")
plt.xticks(df_spec.columns[::100]) # show every 100th date on x-axis ticks
plt.xticks(rotation=75)

plt.show()

Zoom in a bit so that we are only looking at frequencies less than 1500 Hz.

In [ ]:
df_temp = df_spec[df_spec.index < 1500]

fig, ax = plt.subplots(1, 1, figsize=(15, 10))

plt.pcolormesh(df_temp.columns, df_temp.index, df_temp, vmax=0.01)

ax.set_ylabel("Frequency (Hz)")
plt.xticks(df_spec.columns[::100])
plt.xticks(rotation=45)

plt.show()

Now let's look at dates closer to the end of the run.

In [ ]:
date_list = sorted(os.listdir(folder_1st))
date_list = date_list[int((1-len(date_list)*0.2/len(date_list))*len(date_list)):] # get the last 20% of dates

In [ ]:
date_list[0]

In [ ]:
folder_1st = folder_raw_data / '1st_test'

# reminder of bearing acceleration channels
# col_names = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5','float' object has no attribute 'DataFrame'df_spec 'b3_ch6', 'b4_ch7', 'b4_ch8']

df_spec, labels_dict = build_spectrogram_df(folder_1st, date_list, channel_name='b3_ch6', start_time=date_list[0])

df_temp = df_spec[df_spec.index < 400]

fig, ax = plt.subplots(1, 1, figsize=(15, 10))

plt.pcolormesh(df_temp.columns, df_temp.index, df_temp, vmax=0.013)

ax.set_ylabel("Frequency (Hz)")
plt.xticks(df_spec.columns[::20])
plt.xticks(rotation=45)

plt.show()

| Rexnord ZA-2115 Charachteristics           |          |          |
| :----------------------------------------- | -------- | -------- |
| Pitch diameter ($P_D$)                     | 2.815 in | 71.5 mm  |
| Rolling element diameter ($B_D$)           | 0.331 in | 8.4 mm   |
| Number of rolling elements per row ($N_B$) | 16       | 16       |
| Contact angle ($\phi$)                    | 15.7°    | 15.7°    |
| Static load                                | 6000 lbs | 26690 N  |
| **Shaft RPM**                              | 2000 RPM | 2000 RPM |

In [ ]:
pdia = 2.815 # pitch diameter (in)
bdia = 0.331 # rolling ball diameter (in)
nb = 16.0 # no. or rolling elements per row
phi = 15.7 * np.pi / 180 # contact angle (in radians)
load = 6000 # load, in lbs
rps = 2000 / 60.0 # rotations per second (Hz)

We now want to calculate what the important frequencies for the bearing are.

The shaft frequency is 33.3 Hz.

Therefore, the fundamental train frequency (or cage failing frequency) is:

$$\text{FTF} = \frac{\text{RPS}}{2} (1 - \frac{B_D}{P_D} \cos{\phi})$$

In [ ]:
ftf = rps/2 * (1 - bdia/pdia * np.cos(phi))
print('FTF =', ftf)

Ball pass frequency of the inner race:

$$\text{BPFI} = \frac{N_B}{2} \text{RPS} (1 + \frac{B_D}{P_D} \cos{\phi})$$

In [ ]:
bpfi = nb/2 * rps * (1 + bdia/pdia * np.cos(phi))
print('BPFI =', bpfi)

Ball pass frequency of the outer race:

$$\text{BPFO} = \frac{N_B}{2} \text{RPS} (1 - \frac{B_D}{P_D} \cos{\phi})$$

In [ ]:
bpfo = nb/2 * rps * (1 - bdia/pdia * np.cos(phi))
print('BPFO =', bpfo)

Ball spin frequency:

$$\text{BSF} = \frac{P_D}{2 B_D} \text{RPS} (1 - (\frac{B_D}{P_D} \cos{\phi})^2)$$

In [ ]:
bsf = pdia/(2*bdia) * rps * (1 - (bdia/pdia * np.cos(phi))**2)
print('BSF =', bsf)

## Build Features

In [ ]:
date_list = sorted(os.listdir(folder_1st))

df_spec, labels_dict = build_spectrogram_df(folder_1st, date_list, channel_name='b3_ch6', start_time=date_list[0])

In [ ]:
bucket_size = 200

df_temp = df_spec.iloc[:10000]
a = np.array(df_temp) # make numpy array

# get the y-axis (frequency values)
y = np.array(df_temp.index)
y = np.max(y.reshape(-1,bucket_size),axis=1)

# get the max value for each bucket
# https://stackoverflow.com/a/15956341/9214620
max_a = np.max(a.reshape(-1,bucket_size,2156),axis=1)

print('shape of max_a array:', np.shape(max_a))

# get the mean value for each bucket
avg_a = np.mean(a.reshape(-1,bucket_size,2156),axis=1)

fig, ax = plt.subplots(1, 1, figsize=(15, 10))
plt.pcolormesh(df_temp.columns, y, max_a)
ax.set_ylabel("Frequency (Hz)")
plt.xticks(df_temp.columns[::200])
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.plot(max_a.T)
plt.show()

## Look at 3rd Test

In [ ]:
folder_3rd = folder_raw_data / '3rd_test'
date_list = sorted(os.listdir(folder_3rd))

# reminder of bearing acceleration channels
# col_names = ['b1_ch1', 'b2_ch2', 'b3_ch3', 'b4_ch4']

col_names = ['b1_ch1', 'b2_ch2', 'b3_ch3', 'b4_ch4']

df_spec, labels_dict = build_spectrogram_df(folder_3rd, date_list, channel_name='b3_ch3', start_time=date_list[0], col_names=col_names)

df_temp = df_spec
# df_temp = df_spec[df_spec.index < 400]

fig, ax = plt.subplots(1, 1, figsize=(15, 10))


plt.pcolormesh(np.array(df_temp), 
               vmax=0.013
              )
# plt.pcolormesh(df_temp.columns, df_temp.index, df_temp, 
#                vmax=0.013
#               )

# ax.set_ylabel("Frequency (Hz)")
# plt.xticks(df_spec.columns[::50])
# plt.xticks(rotation=45)

plt.show()

In [ ]:
folder_3rd = folder_raw_data / '3rd_test'
date_list = sorted(os.listdir(folder_3rd))

# reminder of bearing acceleration channels
# col_names = ['b1_ch1', 'b2_ch2', 'b3_ch3', 'b4_ch4']

col_names = ['b1_ch1', 'b2_ch2', 'b3_ch3', 'b4_ch4']

df_spec, labels_dict = build_spectrogram_df(folder_3rd, date_list, channel_name='b1_ch1', start_time=date_list[0], col_names=col_names)

df_temp = df_spec
# df_temp = df_spec[df_spec.index < 400]

fig, ax = plt.subplots(1, 1, figsize=(15, 10))


plt.pcolormesh(np.array(df_temp), 
               vmax=0.013
              )
# plt.pcolormesh(df_temp.columns, df_temp.index, df_temp, 
#                vmax=0.013
#               )

# ax.set_ylabel("Frequency (Hz)")
# plt.xticks(df_spec.columns[::50])
# plt.xticks(rotation=45)

plt.show()

In [ ]:
x = []
for t in labels_dict.keys():
    x.append(labels_dict[t][-1])

x = np.sort(np.array(x))

t = x[-1]
print('run-time-to-failure:', f'{t:.3f} days')

## WeiBayes Plot

![weibayes](weibayes.png)

Calculate the $\eta$.

We will assume a $\beta$ of 2.0

Use the 2nd test data.

In [ ]:
folder_2nd = folder_raw_data / '2nd_test'
date_list = sorted(os.listdir(folder_2nd))

# reminder of bearing acceleration channels
# col_names = ['b1_ch1', 'b2_ch2', 'b3_ch3', 'b4_ch4']

col_names = ['b1_ch1', 'b2_ch2', 'b3_ch3', 'b4_ch4']

df_spec, labels_dict = build_spectrogram_df(folder_2nd, date_list, channel_name='b1_ch1', start_time=date_list[0], col_names=col_names)

df_temp = df_spec
# df_temp = df_spec[df_spec.index < 400]

fig, ax = plt.subplots(1, 1, figsize=(15, 10))

plt.pcolormesh(df_temp.columns, df_temp.index, df_temp, 
               vmax=0.013
              )

ax.set_ylabel("Frequency (Hz)")
plt.xticks(df_spec.columns[::50])
plt.xticks(rotation=45)

plt.show()

In the 2nd test set, there are four (4) bearings. They are run until one failes (bearing 1).

We need to find the total run-time-to-failure.

In [ ]:
x = []
for t in labels_dict.keys():
    x.append(labels_dict[t][-1])

x = np.sort(np.array(x))

t = x[-1]
print('run-time-to-failure:', f'{t:.3f} days')

In [ ]:
beta = 2.0 # shape parameter
r = 1 # number of failed bearings
i = 4 # number of bearings

t_array = np.repeat(t, i) # build a time array of t
t_array

"The characteristic life, $\eta$ (eta), is defined as the age at which 63.2% of the units will have failed, the B63.2 life." -- from The New Weibull Handbook

In [ ]:
# characteristic life
eta = (np.sum(t_array ** beta) / 1) ** (1 / beta)
print(eta)

Now with the characteristic life, we can build a Weibull distribution.

In [ ]:
import seaborn as sns

pal = sns.cubehelix_palette(6, rot=-.25, light=.7)

In [ ]:
def weibull_pdf(t, eta, beta):
    "weibull PDF function"
    return (beta/(eta ** beta))*(t**(beta-1.0))*np.exp(-1.0*((t/eta)**beta))

In [ ]:
t = np.linspace(0,50,1000)
f = weibull_pdf(t, eta, beta)

fig, axes = plt.subplots(1, 1, figsize=(4,3), dpi=150)
axes.title.set_text('Weibull Distribution PDF')
axes.set_xlabel('Time (t)')
axes.set_ylabel('F(t)')
axes.grid(False)
axes.axes.xaxis.set_ticks([])
axes.axes.yaxis.set_ticks([])

plt.plot(t, f, color=pal[5], linewidth=2)
# plt.savefig('weibull.png',format='png')

In [ ]:
def weibull_cdf(t, eta, beta):
    "weibull CDF function"
    return (1.0 - np.exp(-1.0*((t/eta)**beta)))

In [ ]:
t = np.linspace(0,50,1000)
F = weibull_cdf(t, eta, beta)

plt.plot(t, F)

## Build Features for 2nd Test

We will now build the features from the 2nd test set. This is what we'll use to train the model on. 

- **Training Set:** 2nd test data
- **Validation Set:** 3rd test data
- **Test Set:** 1st test data

In [ ]:
# folder_2nd = folder_raw_data / '2nd_test'
# date_list = sorted(os.listdir(folder_2nd))

# folder_3rd = folder_raw_data / '3rd_test'
# date_list = sorted(os.listdir(folder_3rd))
# col_names = ['b1_ch1', 'b2_ch2', 'b3_ch3', 'b4_ch4']

folder_1st = folder_raw_data / '1st_test'
date_list = sorted(os.listdir(folder_1st))
col_names = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5','b3_ch6', 'b4_ch7', 'b4_ch8']
# df_spec, labels_dict = build_spectrogram_df(folder_1st, date_list, channel_name='b3_ch6', start_time=date_list1[0], col_names=col_names)

# reminder of bearing acceleration channels
# col_names = ['b1_ch1', 'b2_ch2', 'b3_ch3', 'b4_ch4']

# col_names = ['b1_ch1', 'b2_ch2', 'b3_ch3', 'b4_ch4']

df_spec, labels_dict = build_spectrogram_df(folder_1st, date_list, channel_name='b3_ch6', start_time=date_list[0], col_names=col_names)
df_spec.shape

In [ ]:
samples = df_spec.shape[1]
samples

In [ ]:
np.arange(0,10.0,1)

In [ ]:
bucket_size = 500

samples = df_spec.shape[1]

df_temp = df_spec.iloc[:10000]
a = np.array(df_temp) # make numpy array
print(np.shape(a))

# get the y-axis (frequency values)
y = np.array(df_temp.index)
y = np.max(y.reshape(-1,bucket_size),axis=1)
y = list(y.round().astype('int')[::2])
y.insert(0,0)

# get the max value for each bucket
# https://stackoverflow.com/a/15956341/9214620
max_a = np.max(a.reshape(-1,bucket_size,samples),axis=1)

print('shape of max_a array:', np.shape(max_a))

# get the mean value for each bucket
avg_a = np.mean(a.reshape(-1,bucket_size,samples),axis=1)

fig, ax = plt.subplots(1, 1, figsize=(15, 10))
# plt.pcolormesh(avg_a)
plt.pcolormesh(max_a)
# plt.pcolormesh(y=df_temp.columns, c=max_a)
# ax.set_ylabel("Frequency (Hz)")

# labels = [item.get_text() for item in ax.get_yticklabels()]
loc = ax.get_yticks()
labels = ax.get_yticklabels()
# labels[1] = 'Testing'
# ax.set_yticklabels(y)
# plt.yticks(loc=np.arange(0,11.0,1), labels=y)
# plt.xticks(list(df_temp.columns[::200]))
# plt.xticks(rotation=45)
plt.xlabel('Sample')
plt.ylabel('Frequency Bucket')
plt.savefig('spectrogram_set1.png',format='png', bbox_inches = "tight")
plt.show()

It looks like the last two samples suddenly drop in value, just before the test is stopped. We should remove these before putting them in the training set.

In [ ]:
# plot the last several runs, from one of the buckets
plt.plot(max_a.T[960:])
plt.show()

In [ ]:
plt.plot(avg_a.T)
plt.show()

In [ ]:
plt.plot(max_a.T)
plt.show()

I think we will input a vector into the NN: the 10 values from bucket $t_n$, using the maximum value from each bucket.

$x$ will be the vector of 10 values. $y$ will be the time value, measured in days.

Create the x_train and y_train sets.

In [ ]:
def create_x_y(df_spec, labels_dict,  bucket_size=1000, print_shape=False):

    samples = df_spec.shape[1]

    df_temp = df_spec.iloc[:10000]
    a = np.array(df_temp) # make numpy array

    # get the max value for each bucket
    # https://stackoverflow.com/a/15956341/9214620
    x = np.max(a.reshape(-1,bucket_size,samples),axis=1).T

    temp_days = []
    for i in labels_dict.keys():
        temp_days.append(labels_dict[i][-1])

    temp_days = np.sort(np.array(temp_days))

    run_time = np.max(temp_days)

    # turn into percentage life left
    y = []
    for i in temp_days:
        y.append([i, i/run_time, run_time-i])

    y = np.array(y)

    # drop the last two values from the x and y, since they seem to be erroneous
    x = x[:len(x)-2]
    y = y[:len(y)-2]
    
    if print_shape == True:
        print('Shape of x:', np.shape(x))
        print('Shape of y:', np.shape(y))
        print('run-time-to-failure:', f'{run_time:.3f} days')
    return x, y   
    

In [ ]:
x, y = create_x_y(df_spec, labels_dict,  bucket_size=1000, print_shape=True)

In [ ]:
y[:,2][:10]

In [ ]:
plt.plot(x)
plt.show()

In [ ]:
# show the last several values in the y array
y[::-1][:5]

In [ ]:
plt.plot(x[930:])
plt.show()

## Data Scaling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle

In [ ]:
X, y = create_x_y(df_spec, labels_dict,  bucket_size=1000)

scaler = StandardScaler()

scaler.fit(X)
X = scaler.transform(X)

In [ ]:
plt.plot(x)
plt.show()

In [ ]:
X, y = create_x_y(df_spec, labels_dict,  bucket_size=1000)

min_max_scaler = MinMaxScaler()
min_max_scaler.fit(X)

X = min_max_scaler.transform(X)

In [ ]:
plt.plot(x)
plt.show()

## Train-Test Split

In [ ]:
# test scaling
x, y = create_x_y(df_spec, labels_dict,  bucket_size=1000)

def get_min_max(x):

    # flatten the input array http://bit.ly/2MQuXZd
    flat_vector = np.concatenate(x).ravel()

    min_val = min(flat_vector)
    max_val = max(flat_vector)

    return min_val, max_val

min_val, max_val = get_min_max(x)

def scaler(x, min_val, max_val, lower_norm_val=0, upper_norm_val=1):
    """Scale the signal between a min and max value
    
    Parameters
    ===========
    x : ndarray
        Signal that is being normalized

    max_val : int or float
        Maximum value of the signal or dataset

    min_val : int or float
        Minimum value of the signal or dataset

    lower_norm_val : int or float
        Lower value you want to normalize the data between (e.g. 0)

    upper_norm_val : int or float
        Upper value you want to normalize the data between (e.g. 1)

    Returns
    ===========
    x : ndarray
        Returns a new array that was been scaled between the upper_norm_val
        and lower_norm_val values

    """

    # https://codereview.stackexchange.com/questions/185785/scale-numpy-array-to-certain-range
    col, row = np.shape(x)
    for i in range(col):
        x[i] = np.interp(x[i], (min_val, max_val), (lower_norm_val, upper_norm_val))
    return x

x = scaler(x, min_val, max_val)

In [ ]:
# get X and y
x, y = create_x_y(df_spec, labels_dict,  bucket_size=1000)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=13)

min_val, max_val = get_min_max(x_train)
x_train = scaler(x_train, min_val, max_val)
x_test = scaler(x_test, min_val, max_val)


# # scale X_train, and apply same trasnformation to X_test
# min_max_scaler = MinMaxScaler()
# min_max_scaler.fit(x_train)
# x_train = min_max_scaler.transform(x_train)
# x_test = min_max_scaler.transform(x_test)

# convert to pytorch tensors
x_train = torch.Tensor(x_train)
x_test = torch.Tensor(x_test)
y_train = torch.Tensor(y_train)
y_test = torch.Tensor(y_test)

In [ ]:
x_train.shape

In [ ]:
y_test.shape

In [ ]:
y_train.shape

## Build Super Simple NN

In [ ]:
# make a super small train/test set in order to 
x, y = create_x_y(df_spec, labels_dict,  bucket_size=1000)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=13)
min_val, max_val = get_min_max(x_train)

index_to_keep = np.array([2,400,980])

x_train = x[index_to_keep]
y_train = y[index_to_keep]
# y_train = y_train[:,1]

x_train = scaler(x_train, min_val, max_val)

x_train = torch.Tensor(x_train)
y_train = torch.reshape(torch.Tensor(y_train[:,1]),(-1,1))

print(x_train.shape)
print(y_train.shape)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=10, out_features=16)
        self.fc2 = nn.Linear(in_features=16, out_features=16)
        self.output = nn.Linear(in_features=16, out_features=1)
 
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.output(x)
        return x
    
    # MSE loss function
    def mse(self, y_hat, y):
        return torch.mean((y_hat - y)**2) # y_hat is the prediction
    
net = Net()
net

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [ ]:
net = Net()

optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
criterion = nn.MSELoss()

epochs = 100
loss_arr = []

for i in range(epochs):
    y_hat = net.forward(x_train)
#     print(y_hat.shape)
    loss = net.mse(y_hat, y_train)
#     loss = criterion(y_hat, y_train)
    loss_arr.append(loss)

    if i % 10 == 0:
        print(f'Epoch: {i} Loss: {loss}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
y_hat

In [ ]:
plt.plot(loss_arr)

In [ ]:
plt.plot(loss_arr)

In [ ]:
preds = []

with torch.no_grad():
    for val in x_train:
        y_hat = net.forward(val)
        preds.append(y_hat)

In [ ]:
preds

## Build Neural Network

Calculate Weibull CDF

In [ ]:
# reload data 
if True:
    folder_2nd = folder_raw_data / '2nd_test'
    date_list2 = sorted(os.listdir(folder_2nd))
    col_names = ['b1_ch1', 'b2_ch2', 'b3_ch3', 'b4_ch4']
    df_spec2, labels_dict2 = build_spectrogram_df(folder_2nd, date_list2, channel_name='b1_ch1', start_time=date_list2[0], col_names=col_names)
    ####
    
    
    folder_3rd = folder_raw_data / '3rd_test'
    date_list3 = sorted(os.listdir(folder_3rd))
    col_names = ['b1_ch1', 'b2_ch2', 'b3_ch3', 'b4_ch4']
    df_spec3, labels_dict3 = build_spectrogram_df(folder_3rd, date_list3, channel_name='b3_ch3', start_time=date_list3[0], col_names=col_names)
    ####
    
    
    folder_1st = folder_raw_data / '1st_test'
    date_list1 = sorted(os.listdir(folder_1st))
    col_names = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5','b3_ch6', 'b4_ch7', 'b4_ch8']
    df_spec1, labels_dict1 = build_spectrogram_df(folder_1st, date_list1, channel_name='b3_ch6', start_time=date_list1[0], col_names=col_names)

In [ ]:
def weibull_cdf(t, eta, beta):
    "weibull CDF function"
    return (1.0 - np.exp(-1.0*((t/eta)**beta)))

In [ ]:
[4]*3

In [ ]:
x2, y2 = create_x_y(df_spec2, labels_dict2,  bucket_size=1000, print_shape=False)
x3, y3 = create_x_y(df_spec3, labels_dict3,  bucket_size=1000, print_shape=False)
t2 = np.max(y2[:,0])
t3 = np.max(y3[:,0])
# bold printout https://stackoverflow.com/a/17303428/9214620
print(f'\033[1mTest 2\033[0m run-time: {t2:.3f} days \t\t\033[1mTest 3\033[0m run-time: {t3:.3f} days')

beta = 2.0 # shape parameter
r = 2 # number of failed bearings
i = 8 # number of bearings

t_array = np.append([t2]*4, [t3]*4) # build a time array of t

# characteristic life
eta = (np.sum(t_array ** beta) / 1) ** (1 / beta)
print(eta)

t = np.linspace(0,200,1000)
F = weibull_cdf(t, eta, beta)

fig, ax = plt.subplots(
    1, 1, figsize=(8,6), 
#     dpi=150, constrained_layout=True
)

plt.plot(t, F)
plt.ylabel('Occurence CDF')
plt.xlabel('Bearing Life (days)')
plt.savefig('cdf.png',format='png')
plt.show()



In [ ]:
t_array

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class Net(nn.Module):
    def __init__(self, feat_in):
        super().__init__()
        
        self.feat_in = feat_in
        
        # define the layers
        # [features_in, hidden_layer1_features, ..., features_out]
        feat_in, h1, h2, h3, feat_out = [self.feat_in , 64, 64, 64, 1]
        
        self.fc1 = nn.Linear(feat_in, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc3 = nn.Linear(h2, h3)
        self.fc4 = nn.Linear(h3, feat_out)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        x = self.fc4(x)
       
        return x
    
    # MSE loss function
    def mse(self, y_hat, y):
        return torch.mean((y_hat - y)**2) # y_hat is the prediction
    
    def rmse(self, y_hat, y):
        return torch.sqrt(torch.mean((y_hat - y)**2)) # y_hat is the prediction
    
    def rmsle(self, y_hat, y):
        return torch.sqrt(torch.mean((torch.log(y_hat+1) - torch.log(y+1))**2))
    
    # Weibull loss function
    def weibull_loss(self, y_hat, y, y_days, lambda_mod, eta, beta):
        '''
        y_hat       : predicted RUL
        y           : true RUL
        y_days      : true age (in days)
        lambda_mod  : lambda modifier
        eta         : characteristic life
        beta        : shape parameter for weibull
        '''
        y_hat_days = (y_days + y) - y_hat
        
        # remove any "inf" values from when divided by zero
        y_hat_days = y_hat_days[torch.isfinite(y_hat_days)]
               
        def weibull_cdf(t, eta, beta):
            "weibull CDF function"
            return (1.0 - torch.exp(-1.0*((t/eta)**beta)))
        
        cdf = weibull_cdf(y_days, eta, beta)
        cdf_hat = weibull_cdf(y_hat_days, eta, beta)
        
        return lambda_mod * torch.sqrt(torch.mean(cdf_hat - cdf)**2)
    
    
net = Net(20)
print(net)

In [ ]:
def fwd_pass(net, x, y, y_days, train=False, loss_func='mse', lambda_mod=1.0, eta=13.0, beta=2.0):
    '''Similar to Sentdex tutorial
    https://pythonprogramming.net/analysis-visualization-deep-learning-neural-network-pytorch/
    '''
    if train:
        net.zero_grad()
                
    y_hat = net(x)
        
    if loss_func == 'mse':
        loss = net.mse(y_hat, y)
    if loss_func == 'rmse':
        loss = net.rmse(y_hat, y)
    if loss_func == 'rmsle':
        loss = net.rmsle(y_hat, y)    
    if loss_func == 'weibull':
        rmsle_loss = net.rmsle(y_hat, y)
        weibull_loss = net.weibull_loss(y_hat, y, y_days, lambda_mod, eta, beta)
#         print(f'RMSLE {rmsle_loss:.3f}, Weibull Loss {weibull_loss:.3f}')
        loss = rmsle_loss + weibull_loss
    if loss_func == 'weibull_only':       
        loss = net.weibull_loss(y_hat, y, y_days, lambda_mod, eta, beta)    

    if train:
        loss.backward()
        optimizer.step()

    return loss

In [ ]:
# import EarlyStopping
from utils import EarlyStopping

In [ ]:
def train(net, x_train, y_train, y_train_days,  
          x_val, y_val, y_val_days, 
          loss_func='mse', batch_size=100, epochs=500, patience=7, 
          lambda_mod=1.0, eta=13.0, beta=2.0, early_stop_delay=20):
    
    df = pd.DataFrame()
    
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=patience, verbose=False, early_stop_delay=early_stop_delay)
    
    for epoch in range(epochs):
        
        # track the training/validation losses during epoch
        train_losses = []
        train_losses_mse = []
        
        #############
        # train model
        #############
        for i in range(0, len(x_train), batch_size):

            # create the batches and send to GPU (or CPU)
            # I don't like how we don't shuffle the data before each epoch
            # will have to look at the data-loader documentation and implement
            batch_x = x_train[i:i+batch_size].to(device)
            batch_y = y_train[i:i+batch_size].to(device)
            batch_y_days = y_train_days[i:i+batch_size].to(device)
            
            # train and calculate the losses
            loss = fwd_pass(net, batch_x, batch_y, batch_y_days, train=True, loss_func=loss_func, lambda_mod=lambda_mod, eta=eta, beta=beta)
            loss_mse = fwd_pass(net, batch_x, batch_y, batch_y_days, train=True, loss_func='mse', lambda_mod=lambda_mod, eta=eta, beta=beta)
            train_losses.append(loss.item())
            train_losses_mse.append(loss_mse.item())

    
        ################
        # validate model
        ################
        val_loss = fwd_pass(net, x_val.to(device), y_val.to(device), y_val_days.to(device), loss_func=loss_func, lambda_mod=lambda_mod, eta=eta, beta=beta)
        val_loss_mse = fwd_pass(net, x_val.to(device), y_val.to(device), y_val_days.to(device), loss_func='mse', lambda_mod=lambda_mod, eta=eta, beta=beta)
        
        loss_avg = np.mean(train_losses)
        loss_avg_mse = np.mean(train_losses_mse)
        
        # save the results to a pandas dataframe
        df = df.append(pd.DataFrame([[epoch+1,loss_avg, val_loss.item(),loss_avg_mse, val_loss_mse.item()]],
                                    columns=['epoch', 'loss', 'val_loss', 'loss_mse', 'val_loss_mse']))
        
        early_stopping(val_loss, net)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
        # print out the epoch, loss, and iteration number every 5th epoch
        if epoch % 200 == 0:
            print(f"Epoch: {epoch} \tLoss: {loss_avg:.4f} \tVal Loss: {val_loss:.4f}")
            
    # load the last checkpoint with the best model
    print('Load best model')
    net.load_state_dict(torch.load('checkpoint.pt'))
    
    df = df.reset_index(drop=True)
            
    return df, net

def test(net, x_test, batch_size=BATCH_SIZE):
    with torch.no_grad():  

        y_hats = []
        
        for i in range(0, len(x_test), BATCH_SIZE):
            batch_x = x_test[i:i+BATCH_SIZE].to(device)
            outputs = net(batch_x)
            y_hats.append(np.array(outputs.cpu()).reshape(-1,1))
    
    return np.concatenate(y_hats)

In [ ]:
# reload data 
if False:
    folder_2nd = folder_raw_data / '2nd_test'
    date_list2 = sorted(os.listdir(folder_2nd))
    col_names = ['b1_ch1', 'b2_ch2', 'b3_ch3', 'b4_ch4']
    df_spec2, labels_dict2 = build_spectrogram_df(folder_2nd, date_list2, channel_name='b1_ch1', start_time=date_list2[0], col_names=col_names)
    ####
    
    
    folder_3rd = folder_raw_data / '3rd_test'
    date_list3 = sorted(os.listdir(folder_3rd))
    col_names = ['b1_ch1', 'b2_ch2', 'b3_ch3', 'b4_ch4']
    df_spec3, labels_dict3 = build_spectrogram_df(folder_3rd, date_list3, channel_name='b3_ch3', start_time=date_list3[0], col_names=col_names)
    ####
    
    
    folder_1st = folder_raw_data / '1st_test'
    date_list1 = sorted(os.listdir(folder_1st))
    col_names = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5','b3_ch6', 'b4_ch7', 'b4_ch8']
    df_spec1, labels_dict1 = build_spectrogram_df(folder_1st, date_list1, channel_name='b3_ch6', start_time=date_list1[0], col_names=col_names)

In [ ]:
# rul_or_percent = 'rul'
rul_or_percent = 'percent'
bucket_size = 500

# get X and y
x2, y2 = create_x_y(df_spec2, labels_dict2,  bucket_size=bucket_size, print_shape=False)
x2 = x2[1:] # get rid of the 'zero' so that we don't have inf in weibull loss func
y2 = y2[1:]

x3, y3 = create_x_y(df_spec3, labels_dict3,  bucket_size=bucket_size, print_shape=False)
x3 = x3[1:] # get rid of the 'zero' so that we don't have inf in weibull loss func
y3 = y3[1:]

x_train = np.append(x2, x3,0)
y_train = np.append(y2, y3, 0)

# x_train, y_train = create_x_y(df_spec2, labels_dict2,  bucket_size=bucket_size)
# x_train = x_train[1:] # get rid of the 'zero' so that we don't have inf in weibull loss func
# y_train = y_train[1:]

x_val, y_val = create_x_y(df_spec1, labels_dict1,  bucket_size=bucket_size)
x_val = x_val[1:]
y_val = y_val[1:]


# convert to pytorch tensors
x_train = torch.Tensor(x_train)
x_val = torch.Tensor(x_val)
print(x_train.shape)

y_train_days = torch.reshape(torch.Tensor(y_train[:,0]),(-1,1))
y_val_days = torch.reshape(torch.Tensor(y_val[:,0]),(-1,1))


if rul_or_percent == 'rul':
    y_train = torch.reshape(torch.Tensor(y_train[:,2]),(-1,1))
    print(y_train.shape)
    y_val = torch.reshape(torch.Tensor(y_val[:,2]),(-1,1))
else:
    y_train = torch.reshape(torch.Tensor(y_train[:,1]),(-1,1))
    print(y_train.shape)
    y_val = torch.reshape(torch.Tensor(y_val[:,1]),(-1,1))
    
print(x_train.shape)
print(x_train.dtype)

In [ ]:
x_train.shape

In [ ]:
net = Net(x_train.shape[1])

# setup to run on GPU (or CPU if GPU not avail.)
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on GPU")
else:
    device = torch.device("cpu")
    print("Running on CPU")
    
    
BATCH_SIZE = 100
EPOCHS = 500
patience = 20
early_stop = 0

# SET TOTAL NUMBER OF EPOCHS FOR ALL EXPERIMENTS


net.to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)

# save the results in a dataframe
# df = train(net, x_train, y_train, y_train_days, x_test, y_test, y_test_days,  'mse', lambda_mod=3.0, eta=eta, beta=beta)
df, net = train(net, x_train, y_train, y_train_days,  
          x_val, y_val, y_val_days, 
            loss_func='mse', batch_size=BATCH_SIZE, epochs=EPOCHS, patience=patience, 
          lambda_mod=2.0, eta=eta, beta=2.0, early_stop_delay=0)
# clear_output(wait=False)

In [ ]:
df.head()

In [ ]:
df['val_loss_mse'].iloc[1]

In [ ]:
def plot_trained_model_results(df, net, x_train, y_train, y_train_days,  x_val, y_val, y_val_days, device, rul_or_percent='percent',
                               loss_func='mse', early_stop_delay=0, save_pic=False, show_pic=True, save_name='results'):
    fig, ax = plt.subplots(
    2, 2, figsize=(14,9), 
    constrained_layout=True
    )
    
    #### SELECTED LOSS FUNCTION
    # in sample loss
    ax[0][0].plot(df['epoch'], df['loss'], label='Loss',linewidth=2, alpha=0.4, color='#d73027')

    # out of sample (validation) loss
    ax[0][0].plot(df['epoch'], df['val_loss'], label='Val Loss',linewidth=2, color='#d73027')
    
    epoch_stopped_on = int(df[df['val_loss']==np.min(df['val_loss'][early_stop_delay:])]['epoch'])
    ax[0][0].axvline(int(df[df['val_loss']==np.min(df['val_loss'][early_stop_delay:])]['epoch']), linestyle='--', color='black',label='Early Stopping Checkpoint', alpha=0.6)

    ax[0][0].legend()
    ax[0][0].set_xlabel('epoch')
    ax[0][0].set_ylabel(f'{loss_func} loss')
    ax[0][0].set_title(f'{loss_func} loss')
    
    #### MSE LOSS FUNCTION
    # in sample loss
    ax[0][1].plot(df['epoch'], df['loss'], label='Loss',linewidth=2, alpha=0.4, color='#d73027')

    # out of sample (validation) loss
    ax[0][1].plot(df['epoch'], df['val_loss_mse'], label='Val Loss',linewidth=2, color='#d73027')
    ax[0][1].axvline(epoch_stopped_on, linestyle='--', color='black',label='Early Stopping Checkpoint', alpha=0.6)

#     ax[0][1].legend(loc='lower right')
    ax[0][1].set_xlabel('epoch')
    ax[0][1].set_ylabel('mse loss')
    ax[0][1].set_title('mse loss')
    
    mse_value = df['val_loss_mse'].iloc[epoch_stopped_on-1]
    # get x and y axis limits
    x_min, x_max = ax[0][1].get_xlim()
    y_min, y_max = ax[0][1].get_ylim()
    
    print_text = f'MSE = {mse_value:.3f}'    
    
    ax[0][1].text((x_max-x_min)*0.5+x_min, y_max-(y_max-y_min)*0.1, print_text, fontsize='large', fontweight='semibold',
                  verticalalignment='center', horizontalalignment='center',
        bbox={'facecolor': 'gray', 'alpha': 0.2, 'pad': 10})
    
    print(df['epoch'].max()*0.1)
    print(ax[0][1].get_ylim())

    #### RUL/PERCENT CURVE
    y_hats = test(net, x_train, 100)
    
    if rul_or_percent == 'rul':
        index_sorted = np.flip(np.array(np.argsort(y_train, 0).reshape(-1)))
        ax[1][0].plot(np.array(y_train)[index_sorted], label='True RUL')
        ax[1][0].plot(y_hats[index_sorted], label='Predicted RUL')
        ax[1][0].set_ylabel('RUL (days)')
        ax[1][0].legend(loc='upper right')
    
    else:
        index_sorted = np.array(np.argsort(y_train, 0).reshape(-1))
        ax[1][0].plot(np.array(y_train)[index_sorted], label='True Life Percentage')
        ax[1][0].plot(y_hats[index_sorted], label='Predicted Life Percentage')
        ax[1][0].set_ylabel('life percentage')
        ax[1][0].legend(loc='lower right')
    
    ax[1][0].set_title('train results')
    
    
    y_hats = test(net, x_val, 100)
    
    if rul_or_percent == 'rul':
        index_sorted = np.flip(np.array(np.argsort(y_val, 0).reshape(-1)))
        ax[1][1].plot(np.array(y_val)[index_sorted], label='True RUL')
        ax[1][1].plot(y_hats[index_sorted], label='Predicted RUL')
        ax[1][1].set_ylabel('RUL (days)')
        ax[1][1].legend(loc='upper right')
    
    else:
        index_sorted = np.array(np.argsort(y_val, 0).reshape(-1))
        ax[1][1].plot(np.array(y_val)[index_sorted], label='True Life Percentage')
        ax[1][1].plot(y_hats[index_sorted], label='Predicted Life Percentage')
        ax[1][1].set_ylabel('life percentage')
        ax[1][1].legend(loc='lower right')
    
    ax[1][1].set_title('validation results')

    if save_pic:
        plt.savefig(f'{save_name}.png',format='png')
    
    if show_pic:
        plt.show()
    else:
        plt.close()


plot_trained_model_results(df, net, x_train, y_train, y_train_days,  x_val, y_val, y_val_days, device,
                               loss_func='weibull', save_pic=True, show_pic=True, save_name='results')


In [ ]:
# color blind colors, from https://bit.ly/3qJ6LYL
# [#d73027, #fc8d59, #fee090, #4575b4]
# [redish, orangeish, yellowish, blueish]
# ['mse', 'physics_loss', approx_constraint_loss, 'combined']


fig, ax = plt.subplots(
    1, 1, figsize=(14,9), 
#     dpi=150, constrained_layout=True
)

# in sample loss
ax.plot(df['epoch'], df['loss'], label='Loss',linewidth=2, alpha=0.4, color='#d73027')

# out of sample (validation) loss
ax.plot(df['epoch'], df['val_loss'], label='Val Loss',linewidth=2, color='#d73027')
plt.axvline(int(df[df['val_loss']==np.min(df['val_loss'][early_stop:])]['epoch']), linestyle='--', color='black',label='Early Stopping Checkpoint', alpha=0.6)

ax.legend(loc='upper left')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Training Curves for Loss Function')
# plt.savefig('mse_loss.png',format='png')
plt.show()

In [ ]:
y_hats = test(net, x_val, BATCH_SIZE)

if rul_or_percent == 'rul':
    index_sorted = np.flip(np.array(np.argsort(y_val, 0).reshape(-1)))
else:
    index_sorted = np.array(np.argsort(y_val, 0).reshape(-1))

fig, ax = plt.subplots(1, 1, figsize=(10,6),)


if rul_or_percent == 'rul':
    ax.plot(np.array(y_val)[index_sorted], label='True RUL')
    ax.plot(y_hats[index_sorted], label='Predicted RUL')
    ax.set_ylabel('RUL (days)')
    ax.legend(loc='upper right')
    
else:
    ax.plot(np.array(y_val)[index_sorted], label='True Life Percentage')
    ax.plot(y_hats[index_sorted], label='Predicted Life Percentage')
    ax.set_ylabel('Life Percentage')
    ax.legend(loc='upper left')


ax.set_xlabel('Sample (from earliest to latest)')
plt.savefig('validation_set_results_set1.png',format='png')
plt.show()

# calculate percentage error
error = torch.absolute(y_val - y_hats)/y_val*100

# average percent error across all predictions
error_avg = torch.mean(error[torch.isfinite(error)])
print(f'Average error: {error_avg:.3f}')

TSS = torch.sum((y_val - torch.mean(y_val))**2)

RSS = torch.sum((torch.tensor(y_hats) - torch.mean(y_val))**2)

r_squared = 1 - RSS/TSS
print(f'R-squared: {r_squared:.3f}')

In [ ]:
x2, y2 = create_x_y(df_spec2, labels_dict2,  bucket_size=bucket_size, print_shape=False)
x2 = x2[1:] # get rid of the 'zero' so that we don't have inf in weibull loss func
y2 = y2[1:]

x_train = scaler(x2, min_val, max_val)
x_train = torch.tensor(x_train).float()
y_train = torch.tensor(y2).float()

y_hats = test(net, x_train, BATCH_SIZE)



if rul_or_percent == 'rul':
    y_train = torch.reshape(torch.Tensor(y_train[:,2]),(-1,1)).float()
    index_sorted = np.flip(np.array(np.argsort(y_train, 0).reshape(-1)))
else:
    y_train = torch.reshape(torch.Tensor(y_train[:,1]),(-1,1)).float()
    index_sorted = np.array(np.argsort(y_train, 0).reshape(-1))

fig, ax = plt.subplots(1, 1, figsize=(10,6),)


if rul_or_percent == 'rul':
    ax.plot(np.array(y_train)[index_sorted], label='True RUL')
    ax.plot(y_hats[index_sorted], label='Predicted RUL')
    ax.set_ylabel('RUL (days)')
    ax.legend(loc='upper right')
    
else:
    ax.plot(np.array(y_train)[index_sorted], label='True Life Percentage')
    ax.plot(y_hats[index_sorted], label='Predicted Life Percentage')
    ax.set_ylabel('Life Percentage')
    ax.legend(loc='upper left')


ax.set_xlabel('Sample (from earliest to latest)')
plt.savefig('train_set_results_set2.png',format='png')
plt.show()
# calculate percentage error
error = torch.absolute(y_train - y_hats)/y_train*100

# average percent error across all predictions
error_avg = torch.mean(error[torch.isfinite(error)])
print(f'Average error: {error_avg:.3f}')

TSS = torch.sum((y_train - torch.mean(y_train))**2)

RSS = torch.sum((torch.tensor(y_hats) - torch.mean(y_train))**2)

r_squared = 1 - RSS/TSS
print(f'R-squared: {r_squared:.3f}')

In [ ]:
x3, y3 = create_x_y(df_spec3, labels_dict3,  bucket_size=bucket_size, print_shape=False)
x3 = x3[1:] # get rid of the 'zero' so that we don't have inf in weibull loss func
y3 = y3[1:]

x_train = scaler(x3, min_val, max_val)
x_train = torch.tensor(x_train).float()
y_train = torch.tensor(y3).float()

y_hats = test(net, x_train, BATCH_SIZE)

if rul_or_percent == 'rul':
    y_train = torch.reshape(torch.Tensor(y_train[:,2]),(-1,1)).float()
    index_sorted = np.flip(np.array(np.argsort(y_train, 0).reshape(-1)))
else:
    y_train = torch.reshape(torch.Tensor(y_train[:,1]),(-1,1)).float()
    index_sorted = np.array(np.argsort(y_train, 0).reshape(-1))

fig, ax = plt.subplots(1, 1, figsize=(10,6),)


if rul_or_percent == 'rul':
    ax.plot(np.array(y_train)[index_sorted], label='True RUL')
    ax.plot(y_hats[index_sorted], label='Predicted RUL')
    ax.set_ylabel('RUL (days)')
    ax.legend(loc='upper right')
    
else:
    ax.plot(np.array(y_train)[index_sorted], label='True Life Percentage')
    ax.plot(y_hats[index_sorted], label='Predicted Life Percentage')
    ax.set_ylabel('Life Percentage')
    ax.legend(loc='upper left')


ax.set_xlabel('Sample (from earliest to latest)')
plt.savefig('train_set_results_set3.png',format='png')
plt.show()
# calculate percentage error
error = torch.absolute(y_train - y_hats)/y_train*100

# average percent error across all predictions
error_avg = torch.mean(error[torch.isfinite(error)])
print(f'Average error: {error_avg:.3f}')

TSS = torch.sum((y_train - torch.mean(y_train))**2)

RSS = torch.sum((torch.tensor(y_hats) - torch.mean(y_train))**2)

r_squared = 1 - RSS/TSS
print(f'R-squared: {r_squared:.3f}')

In [ ]:
net = Net(x_train.shape[1])

# setup to run on GPU (or CPU if GPU not avail.)
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on GPU")
else:
    device = torch.device("cpu")
    print("Running on CPU")
    
    
BATCH_SIZE = 100

# SET TOTAL NUMBER OF EPOCHS FOR ALL EXPERIMENTS
EPOCHS_ALL = 2000

EPOCHS = EPOCHS_ALL

net.to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)

# save the results in a dataframe
df = train(net, x_train, y_train, y_train_days, x_test, y_test, y_test_days,  'rmsle', lambda_mod=3.0, eta=eta, beta=beta)
# df = train(net, x_train, y_train, y_train_days, x_test, y_test, y_test_days,  'weibull', lambda_mod=3.0, eta=eta, beta=beta)
clear_output(wait=False)

In [ ]:
x_val = x_train
y_val = y_train

y_hats = test(net, x_val, BATCH_SIZE)

index_sorted = np.argsort(y_val, 0).reshape(-1)
index_sorted.shape

plt.plot(np.array(y_val)[index_sorted])
plt.plot(y_hats[index_sorted])
plt.show()

# calculate percentage error
error = torch.absolute(y_val - y_hats)/y_val*100

# average percent error across all predictions
error_avg = torch.mean(error[torch.isfinite(error)])
print(f'Average error: {error_avg:.3f}')

TSS = torch.sum((y_val - torch.mean(y_val))**2)

RSS = torch.sum((torch.tensor(y_hats) - torch.mean(y_val))**2)

r_squared = 1 - RSS/TSS
print(f'R-squared: {r_squared:.3f}')

In [ ]:
1 - RSS/TSS

In [ ]:
pred_val = y_hats[index_sorted][1:][:,0]
real_val = np.array(y_test)[index_sorted][1:][:,0]
print(pred_val.shape)
print(real_val.shape)

In [ ]:
np.mean(np.absolute(real_val-pred_val)/real_val)

In [ ]:
plt.plot(pred_val)

In [ ]:
plt.plot(real_val)

In [ ]:
np.absolute(np.array(y_test)[index_sorted][1:] - y_hats[index_sorted][1:])/(np.array(y_test)[index_sorted][1:])*100

In [ ]:
y_hats[index_sorted]

In [ ]:
torch.mean(error[torch.isfinite(error)])

In [ ]:
a = (torch.absolute(y_test - y_hats)/y_test)


In [ ]:
(32 - (32-33))/32

In [ ]:
np.absolute(32 -31)/32

In [ ]:
y_test[22]

In [ ]:
y_hats[22]

In [ ]:
torch.mean(a[0:23])

In [ ]:
a[0:23]

In [ ]:
c, d =torch.where(a<0)
c

In [ ]:
torch.mean(acc[torch.isfinite(acc)])

In [ ]:
np.mean(acc[:,0])

In [ ]:
#Weibull 20 input size, lambda=3
y_hats = test(net, x_train, BATCH_SIZE)

index_sorted = np.argsort(y_train, 0).reshape(-1)
index_sorted.shape

plt.plot(np.array(y_train)[index_sorted])
plt.plot(y_hats[index_sorted])

In [ ]:
#Weibull 20 input size, lambda=1
y_hats = test(net, x_test, BATCH_SIZE)

index_sorted = np.argsort(y_test, 0).reshape(-1)
index_sorted.shape

plt.plot(np.array(y_test)[index_sorted])
plt.plot(y_hats[index_sorted])

In [ ]:
#Weibull 20 input size, lambda=1
y_hats = test(net, x_train, BATCH_SIZE)

index_sorted = np.argsort(y_train, 0).reshape(-1)
index_sorted.shape

plt.plot(np.array(y_train)[index_sorted])
plt.plot(y_hats[index_sorted])

In [ ]:
#RMSLE 20 input size
plt.plot(np.array(y_train)[index_sorted])
plt.plot(y_hats[index_sorted])

In [ ]:
#RMSLE
plt.plot(np.array(y_test)[index_sorted])
plt.plot(y_hats[index_sorted])

In [ ]:
#RMSE
plt.plot(np.array(y_test)[index_sorted])
plt.plot(y_hats[index_sorted])

In [ ]:
#MSE
plt.plot(np.array(y_test)[index_sorted])
plt.plot(y_hats[index_sorted])

In [ ]:
net = Net()

# setup to run on GPU (or CPU if GPU not avail.)
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on GPU")
else:
    device = torch.device("cpu")
    print("Running on CPU")
    
    
BATCH_SIZE = 50

# SET TOTAL NUMBER OF EPOCHS FOR ALL EXPERIMENTS
EPOCHS_ALL = 1000

EPOCHS = EPOCHS_ALL

net = Net()
net.to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)

# save the results in a dataframe
df = train(net, x_train, y_train, y_train_days, x_test, y_test, y_test_days,  'weibull', lambda_mod=1.0, eta=eta, beta=beta)
clear_output(wait=False)

In [ ]:
index_sorted = np.argsort(y_test, 0).reshape(-1)
index_sorted.shape

plt.plot(np.array(y_test)[index_sorted])
plt.plot(y_hats[index_sorted])

In [ ]:
plt.plot(np.array(y_test)[index_sorted])
plt.plot(y_hats[index_sorted])

In [ ]:
plt.plot(np.array(y_test)[index_sorted])
plt.plot(y_hats[index_sorted])

In [ ]:
fig, ax = plt.subplots(
    1, 1, figsize=(14,9), 
#     dpi=150, constrained_layout=True
)

ax.plot(x, y[:,1], color='grey',label='True Signal\n(no noise)', linewidth=5, alpha=0.3)
# ax.scatter(x_test, y_hats, s=7, label='Predictions', color='#d73027', alpha=0.7 )
# ax.scatter(x_train, y_train_noisy, s=20, alpha=0.8, label='Training Data',linewidths=0, marker='D')
ax.legend(loc='lower left')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Predictions using MSE Loss Function')
# plt.savefig('mse_predictions.png',format='png')
plt.show()

In [ ]:
y_hats

![pdf_cdf](pdf_cdf.png)

In [ ]:
def weibull(t, eta, beta):
    "weibull PDF function"
    return (beta/(eta ** beta))*(t**(beta-1.0))*np.exp(-1.0*((t/eta)**beta))

In [ ]:
eta = 1000.0
beta = 5.0
c = eta
k = beta

t = np.linspace(0,2000,500)

f = weibull(t, c, k)
plt.plot(t, f)

## Practice Weiubull Fitting

Get data.

In [ ]:
bucket_size = 400

df_temp = df_spec.iloc[:10000]
a = np.array(df_temp) # make numpy array

# get the y-axis (frequency values)
y = np.array(df_temp.index)
y = np.max(y.reshape(-1,bucket_size),axis=1)

# get the max value for each bucket
# https://stackoverflow.com/a/15956341/9214620
max_a = np.max(a.reshape(-1,bucket_size,2156),axis=1)

print('shape of max_a array:', np.shape(max_a))

# get the mean value for each bucket
avg_a = np.mean(a.reshape(-1,bucket_size,2156),axis=1)

fig, ax = plt.subplots(1, 1, figsize=(15, 10))
plt.pcolormesh(max_a)
plt.show()

In [ ]:
max_a.shape

In [ ]:
plt.plot(max_a[10])
plt.show()

In [ ]:
# make 10th bucket the x value
x = []
for t in labels_dict.keys():
    x.append(labels_dict[t][-1])

x = np.sort(np.array(x))
# x = np.sort(np.array(x)[::20])
print(np.shape(x))

In [ ]:
y = max_a[10]
# y = max_a[10][::20]
print(np.shape(y))

In [ ]:
plt.plot(x, y)
plt.ylabel("Acceleration")
plt.xlabel("Days")
plt.show()

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def exponential(x, a, k, b):
    return a*np.exp(x*k) + b

In [ ]:
popt_exponential, pcov_exponential = curve_fit(exponential, x, y)

In [ ]:
plt.plot(x, exponential(x, *popt_exponential))
plt.plot(x, y)

In [ ]:
# http://www.unm.edu/~mr369/python/data-analysis.html
def weibull(x, c, k, a):
    "weibull to fit"
    return a+(k/c)*((x/c)**(k-1.0))*np.exp(-1.0*((x/c)**k))

In [ ]:
3 * 10 ** 1

In [ ]:
popt, pcov = curve_fit(weibull, x, y,bounds=(0, [100.0, 30.0, 0.2]))
popt

In [ ]:
popt[0]

In [ ]:
plt.plot(x, y)
plt.plot(x, weibull(x, *popt))

In [ ]:
def ufrf(x, n, b, Y, K):
    "universal failure rate function to fit"
    return Y + K * (b/(n ** b))*x**(b-1)

In [ ]:
# scale between 0 and 1
y = (y - np.min(y))/np.max(y)

In [ ]:
popt, pcov = curve_fit(ufrf, x, y,
                       bounds=(0.0001, [100.0, 20, 10.0, 10.0])
                      )
popt

In [ ]:
plt.plot(x, y)
plt.plot(x, ufrf(x, *popt))


$h(t) = (\beta / \eta)(t/ \eta)^{\beta - 1}$ - Weibull hazard function (from 2-7 in The New Weibull Handbook)

In [ ]:
def weibull_hazard(t, n, b):
    return (b / n) * (t / n) ** (b -1)

In [ ]:
plt.plot(x, y)
plt.plot(x, weibull_hazard(x, 38, 20))

In [ ]:
plt.plot(x, 1-np.exp(-x/2))

In [ ]:
beta, n, Y, K = popt

In [ ]:
wc = Y + K * (beta)/(np.power(n, beta)) * np.power(x, (beta - 1))

In [ ]:
wc

In [ ]:
beta, n, Y, K = (5.0, 100.0, 1.9769, 0.000355)

In [ ]:
ans = func(x, beta, n, Y, K)

In [ ]:
plt.plot(x, ans)

In [ ]:
ans

In [ ]:
func(x, *popt)

In [ ]:
plt.plot(x, func(x, *popt))

In [ ]:
# https://stackoverflow.com/questions/3433486/how-to-do-exponential-and-logarithmic-curve-fitting-in-python-i-found-only-poly
a = np.polyfit(x,np.log(y),1)
print(a)

In [ ]:
plt.plot(x, y)
plt.plot(x, np.exp(a[0])*np.exp(a[1]*x))
plt.ylabel("Acceleration")
plt.xlabel("Days")
plt.show()

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html


In [ ]:
x = np.array([10, 19, 30, 35, 51])
y = np.array([1, 7, 20, 50, 79])

plt.plot(x,y)

 y = AeBx

In [ ]:
np

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
plt.pcolormesh(avg_a)
plt.show()

## Make Spectrogram in Plotly

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio

In [ ]:
fig = go.Figure(data=go.Heatmap(
        z=df_spec,
        x=date_list,
        y=df_spec.index,
        colorscale='Viridis'))

fig.update_layout(
    xaxis_nticks=2)

pio.write_html(fig,file='test.html',auto_open=True)

In [ ]:
########

In [ ]:
date_list = sorted(os.listdir(folder_1st))
date_list = date_list[int((1-len(date_list)*0.2/len(date_list))*len(date_list)):] # get the last 20% of dates

In [ ]:
folder_1st = folder_raw_data / '1st_test'

# reminder of bearing acceleration channels
# col_names = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5', 'b3_ch6', 'b4_ch7', 'b4_ch8']

df_spec, labels_dict = build_spectrogram_df(folder_1st, date_list, channel_name='b3_ch6', start_time=date_list[0])


In [ ]:
df_spec = df_spec * 10**6
df_spec.head()

In [ ]:
df_spec = df_spec.astype(dtype='int')
df_spec.head()

In [ ]:
date_list = sorted(os.listdir(folder_1st))
date_list = date_list[int((1-len(date_list)*0.2/len(date_list))*len(date_list)):] # get the last 20% of dates

In [ ]:



fig = go.Figure(data=go.Heatmap(
        z=df_spec,
        x=df_spec.columns,
        y=df_spec.index,
        colorscale='Viridis'))

fig.update_layout(
    xaxis_nticks=2)

pio.write_html(fig,file='test.html',auto_open=True)

In [ ]:
import plotly.graph_objects as go
import datetime
import numpy as np
np.random.seed(1)

programmers = ['Alex','Nicole','Sara','Etienne','Chelsea','Jody','Marianne']

base = datetime.datetime.today()
dates = base - np.arange(180) * datetime.timedelta(days=1)
z = np.random.poisson(size=(len(programmers), len(dates)))

fig = go.Figure(data=go.Heatmap(
        z=z,
        x=dates,
        y=programmers,
        colorscale='Viridis'))

fig.update_layout(
    title='GitHub commits per day',
    xaxis_nticks=36)

fig.show()

In [ ]:
# # rolling average
# df_spec = df_spec.rolling(25,axis=1).mean()
# df_spec = df_spec.dropna(axis=1)
# df_spec.head()